In [1]:
# Cell 1: 强制安装加速库到当前环境
!pip install -q xformers==0.0.29.post1 diffusers==0.36.0 transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# Cell 1: 暴力重装，确保版本百分之百匹配
!pip uninstall -y torch torchvision torchaudio xformers
!pip install -q torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 xformers==0.0.28.post1 --index-url https://download.pytorch.org/whl/cu121
!pip install -q diffusers==0.30.0 transformers accelerate

Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: xformers 0.0.29.post1
Uninstalling xformers-0.0.29.post1:
  Successfully uninstalled xformers-0.0.29.post1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.9/798.9 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.6 MB/s eta 

In [1]:
# Cell 2: 验证环境并跑分
import torch
import time, statistics
from diffusers import StableDiffusionPipeline

print("=== 正在检查环境 ===")
try:
    import xformers
    print(f"✅ xFormers 加速库已就绪: {xformers.__version__}")
except Exception as e:
    print(f"❌ xFormers 依然有问题: {e}")

model_id = "runwayml/stable-diffusion-v1-5"
prompt = "a cinematic photo of a corgi astronaut"
device = "cuda"

def get_latency(use_xf=False):
    # 每次清理显存，保证公平
    torch.cuda.empty_cache()

    pipe = StableDiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        safety_checker=None  # 关掉安全检查，让测试更纯粹
    ).to(device)

    if use_xf:
        pipe.enable_xformers_memory_efficient_attention()

    print(f"[{'xFormers' if use_xf else 'Baseline'}] 预热中...")
    for _ in range(2): pipe(prompt, num_inference_steps=20)

    print(f"[{'xFormers' if use_xf else 'Baseline'}] 测速中...")
    times = []
    for _ in range(5):
        start = time.perf_counter()
        pipe(prompt, num_inference_steps=20)
        times.append(time.perf_counter() - start)

    del pipe # 释放显存
    return statistics.mean(times)

# 执行对比
time_base = get_latency(use_xf=False)
time_xf = get_latency(use_xf=True)

print(f"\n" + "="*40)
print(f"🏁 Baseline (原生): {time_base:.2f}s")
print(f"🏁 xFormers (加速): {time_xf:.2f}s")
print(f"🚀 性能提升: {((time_base/time_xf)-1)*100:.1f}%")
print("="*40)

=== 正在检查环境 ===
✅ xFormers 加速库已就绪: 0.0.28.post1


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]


🏁 Baseline (原生): 3.25s
🏁 xFormers (加速): 3.18s
🚀 性能提升: 2.3%
